# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Transfer Learning com TensorFlow e Inception V3

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


A Visão Computacional evoluiu rapidamente na última década devido a algumas tendências que se cruzam:

  - Avanços na pesquisa em aprendizado de máquina
  - Petabytes de dados de imagem disponíveis online
  - Processamento paralelo em GPU

E uma técnica que vem sendo cada vez mais utilizada é a **Transferência de Aprendizado**. Por que não usar um modelo pré-treinado (ou pelo menos algumas camadas de um modelo pré-treinado) e assim aumentar a capacidade de previsão do seu modelo? Isso é Transfer Learning. Transferir o aprendizado de um modelo para outro.

Em vez de treinar um modelo a partir do zero, podemos importar os pesos de outro modelo como base para o nosso caso de uso específico.

**InceptionV3**

O InceptionV3 é um dos modelos mais avançados de Visão Computacional atualmente disponíveis. Criado pelos membros do Google Research, é um modelo pré-treinado de muitas camadas ocultas e treinado com uma variedade de imagens. O gráfico abaixo mostra a arquitetura complexa dessa rede realmente profunda.

![](imagens/inceptionv3.png)

Aqui o link do paper de pesquisa original: https://arxiv.org/pdf/1512.00567.pdf

Neste exemplo vamos usar o TensorFlow para importar um modelo InceptionV3 pré-treinado, extrair algumas camadas e treinar nosso próprio modelo.

## Dataset Para Transfer Learning com InceptionV3

Usaremos o conjunto de dados Caltech Birds, para treinar um modelo capaz de prever as espécies de pássaros retratados em uma fotografia. 


Podemos aproveitar o modelo Inception V3, que já foi treinado em milhares de imagens, e modificá-lo para um caso de uso específico.

### Carregando os Dados

In [2]:
!nvidia-smi

Sun May  3 19:46:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 23%   38C    P8     9W / 250W |    114MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   35C    P8     9W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

O pacote tensorflow_datasets traz o dataset Caltech Birds. Vamos instalar o pacote e então carregar o dataset.

In [1]:
# Instala o pacote
!pip3 install -q tensorflow_datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboardx 2.5.1 requires protobuf<=3.20.1,>=3.8.0, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
onnx 1.12.0 requires protobuf<=3.20.1,>=3.12.2, but you have protobuf 3.20.3 which is incompatible.


In [2]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds

AttributeError: '_LazyImporter' object has no attribute 'is_np_dtype'

In [5]:
# Carregando o dataset
# O download pode levar algum tempo, pois o dataset tem mais de 1 GB
(df_treino, df_teste), ds_info = tfds.load('caltech_birds2011',
                                           split = ['train', 'test'],
                                           shuffle_files = True,
                                           as_supervised = True,
                                           with_info = True)

In [6]:
# Print
print("\nResumo do Dataset:\n", ds_info.description)


Resumo do Dataset:
 Caltech-UCSD Birds 200 (CUB-200) is an image dataset with photos 
of 200 bird species (mostly North American). The total number of 
categories of birds is 200 and there are 6033 images in the 2010 
dataset and 11,788 images in the 2011 dataset.
Annotations include bounding boxes, segmentation labels.


### Pré-Processamento de Dados 

In [7]:
# Hiperparâmetros
batch_size = 64
image_height = 400
image_width = 400

In [8]:
# Função para normalização das imagens
def norm(image, label):
    return tf.cast(image, tf.float32) / 255., label

In [9]:
# Função para pad (ajuste do tamanho das imagens)
def pad(image, label):
    return tf.image.resize_with_pad(image, image_height, image_width), label

In [10]:
# Prepara os dados de treino
df_treino = df_treino.map(norm, num_parallel_calls = tf.data.experimental.AUTOTUNE)
df_treino = df_treino.map(pad, num_parallel_calls = tf.data.experimental.AUTOTUNE)
df_treino = df_treino.batch(batch_size)
df_treino = df_treino.prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
# Shape
df_treino

<PrefetchDataset shapes: ((None, 400, 400, 3), (None,)), types: (tf.float32, tf.int64)>

In [12]:
# Prepara os dados para validação
df_teste = df_teste.map(norm, num_parallel_calls = tf.data.experimental.AUTOTUNE)
df_teste = df_teste.map(pad, num_parallel_calls = tf.data.experimental.AUTOTUNE)
df_teste = df_teste.batch(batch_size)
df_teste = df_teste.prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
# Shape
df_teste

<PrefetchDataset shapes: ((None, 400, 400, 3), (None,)), types: (tf.float32, tf.int64)>

### Carregando o Modelo InceptionV3 Para Transfer Learning

In [14]:
# Carrega a função InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [15]:
# Carrega o modelo pré-treinado
# Observe que estamos usando os pesos imagenet, de um grande conjunto de imagens
pretrained_model = InceptionV3(input_shape = (image_height,image_width, 3), include_top = False, weights = 'imagenet')

In [16]:
# O loop abaixo percorre todas as camadas do modelo pré-treinado e altera a propriedade para "não-treinável"
# Queremos treinar nossas próprias camadas, não as camadas do Inception (que já foram treinadas)
for layer in pretrained_model.layers:
    layer.trainable = False

In [17]:
# Nivelamos a saída e adicionamos uma camada totalmente conectada com um nó para cada classe
x = tf.keras.layers.Flatten()(pretrained_model.output)
x = tf.keras.layers.Dense(200, activation = 'softmax')(x)

In [18]:
# Agora definimos nossa própria camada como camada de entrada
model = tf.keras.Model(pretrained_model.input, x)

In [19]:
# E então compilamos o modelo
model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer = tf.keras.optimizers.Adam(0.001), 
              metrics = ['accuracy'],)

In [20]:
# Sumário do modelo (o modelo é imenso, com mais de 21 milhões de parâmetros treináveis)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 199, 199, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 199, 199, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 199, 199, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Treinamento

Por fim, treinamos o modelo! E é isso. Transfer Learning é uma forma rápida e fácil de ter um modelo de alta precisão para tarefas de Visão Computacional com Deep Learning.

In [21]:
# Treinamento
history = model.fit(df_treino, epochs = 50, shuffle = False)

Epoch 1/50
94/94 [==============================] - 25s 262ms/step - loss: 73.8974 - accuracy: 0.1363
Epoch 2/50
94/94 [==============================] - 16s 168ms/step - loss: 4.8571 - accuracy: 0.8328
Epoch 3/50
94/94 [==============================] - 16s 168ms/step - loss: 2.0601 - accuracy: 0.9067
Epoch 4/50
94/94 [==============================] - 16s 168ms/step - loss: 1.4863 - accuracy: 0.9278
Epoch 5/50
94/94 [==============================] - 16s 170ms/step - loss: 1.3834 - accuracy: 0.9381
Epoch 6/50
94/94 [==============================] - 16s 170ms/step - loss: 2.0169 - accuracy: 0.9281
Epoch 7/50
94/94 [==============================] - 16s 170ms/step - loss: 2.6746 - accuracy: 0.9173
Epoch 8/50
94/94 [==============================] - 16s 170ms/step - loss: 2.7469 - accuracy: 0.9188
Epoch 9/50
94/94 [==============================] - 16s 172ms/step - loss: 2.9482 - accuracy: 0.9168
Epoch 10/50
94/94 [==============================] - 16s 173ms/step - loss: 2.7984 - accur

Modelo treinado e com excelente performance!

# Fim